# Inhibitor effect on celltype

In [1]:
# If problems installing slingshot
# sudo apt-get update
# sudo apt-get install libglu1-mesa-dev 
# sudo apt-get install freeglut3-dev
# sudo apt-get install mesa-common-dev
# then
# install.packages("rgl")
# BiocManager::install("slingshot")
library(dplyr)
library(RColorBrewer)
library(destiny, quietly = TRUE)
library(ggplot2)
library(pheatmap)
library(cowplot)
library(reshape2)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



********************************************************

Note: As of version 1.0.0, cowplot does not change the

  default ggplot2 theme anymore. To recover the previous

  behavior, execute:
  theme_set(theme_cowplot())

********************************************************




In [2]:
sessionInfo()

R version 4.0.0 (2020-04-24)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.3 LTS

Matrix products: default
BLAS/LAPACK: /home/jovyan/my-conda-envs/R4/lib/libopenblasp-r0.3.9.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] reshape2_1.4.4     cowplot_1.0.0      pheatmap_1.0.12    ggplot2_3.3.1     
[5] destiny_3.2.0      RColorBrewer_1.1-2 dplyr_1.0.0       

loaded via a namespace (and not attached):
 [1] bitops_1.0-6                matrixStats_0.56.0         
 [3] xts_0.12-0                  GenomeInfoDb_1.24.0        

# Process with Seurat from raw

In [6]:
metadata = read.csv('data/scATAC/seurat.output-S2D19/data/barcode_metadata.csv', stringsAsFactors = F)
names(metadata)[1] = 'index'
names(metadata)[ names(metadata) == 'inhibitor' ] = 'Inhibition'
names(metadata)[ names(metadata) == 'day' ] = 'Days'
names(metadata)[ names(metadata) == 'celltype' ] = 'clusters'

In [9]:
create_contingency_table = function(perturbed_cl, control_cl, perturbed, control){
    if(length(perturbed_cl) == 0 || length(control_cl) == 0){
        perturbed_cl = c(perturbed_cl, '1')
        control_cl = c(control_cl, '1')
        perturbed = c(perturbed, '1')
        control = c(control, '1')
    }
    m = matrix(c(length(perturbed_cl), length(control_cl), 
               length(perturbed),  length(control)), 
             ncol = 2, dimnames = list(c('I', 'C'), c('in_cl', 'total')) )
  return(m)  
}


enrichment = function(perturbed_cl, control_cl, perturbed, control){
  funtable = create_contingency_table(perturbed_cl, control_cl, perturbed, control)
  out = NULL
#   if ( funtable[1,1] > 1 ){
    ft = fisher.test(funtable)
    pvalue = ft$p.value
    estimate = ft$estimate
    conf.int = ft$conf.int
    out = data.frame(pvalue= pvalue, odds.ratio = estimate, min_confint = conf.int[1], max_confint = conf.int[2],  stringsAsFactors = F)
  return(out)
#   }
}

test_cluster = function(cl){
    message(cl)
    results = list()
    for (I in inhibitors)
        for (tm in times){
            perturbed = subset(metadata, Inhibition == I & Days == tm)$index
            perturbed_cl = subset(metadata, Inhibition == I & Days == tm & clusters == cl)$index
            control = subset(metadata, Inhibition == 'N_I' & Days == tm)$index
            control_cl = subset(metadata, Inhibition == 'N_I' & Days == tm & clusters == cl)$index
            out = enrichment(perturbed_cl, control_cl, perturbed, control)
            if( is.null(out))
                next()
            out$inhibitor2 = I
            out$inhibitor = I
            out$id = paste0(out$inhibitor , ' day', tm)
            out$cell_type = cl
            results[[paste0(I, '_day', tm) ]] = out

        }
    df = melt(results, id.vars = colnames(results[[1]]))[, -9]
    df$pvalue_adj = p.adjust(df$pvalue, method = 'bonferroni')
    df$label = ifelse(df$pvalue_adj < 0.05, '*', '')
    df$label[ df$pvalue_adj < 0.01] = '**'
    df$label[ df$pvalue_adj < 0.001] = '***'
    print(df)
    
    y_max = max(df$max_confint[ ! is.infinite(df$max_confint) ] )
    y_max = log(y_max) + 0.7
    P = ggplot(df, aes(x=id, y=log(odds.ratio), fill = inhibitor, label = label) ) + 
       # scale_y_continuous(limits = log(c(min(df$min_confint[ ! is.infinite(df$min_confint) ]), max(df$max_confint[ ! is.infinite(df$max_confint) ] ) )) ) + 
        scale_y_continuous(limits = c(log(min(df$min_confint[ ! is.infinite(df$min_confint) ])), y_max) ) + 
        geom_bar(stat = 'identity') + 
        geom_point(show.legend = F) + 
        geom_errorbar(mapping = aes(ymin = log(min_confint), ymax = log(max_confint)), width=0.5, size=0.2) +
        geom_text(color = 'black', y = y_max*0.9, size = 8) +
        scale_fill_manual(values = c("#6c94d9", "#e39696") ) +
        theme_bw(18) + theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust = 1), 
                             panel.grid.major = element_blank(), panel.grid.minor = element_blank()) + 
        ggtitle(cl) + xlab('') + ylab('odds ratio (log scale)\n N cells enrichment')
    P
    
    ggsave(filename = paste0('data/scATAC/seurat.output-S2D19/general/barplot_inhibitorsenrichment_', cl, '.pdf') )
    return(P)
}

In [ ]:
inhibitors = setdiff(metadata$Inhibition, 'N_I')
times = c(3, 7)
Re = list()
for (cl in unique(metadata$clusters) )
    try(Re[[cl]] <- test_cluster(cl))
Re

proliferative



        pvalue odds.ratio min_confint max_confint inhibitor2 inhibitor
1 7.168560e-01  0.9747415   0.8547747    1.111646   NOTCH-in  NOTCH-in
2 3.635628e-14  0.6181130   0.5417407    0.703795   NOTCH-in  NOTCH-in
3 6.710961e-01  1.0287717   0.9031718    1.171980     WNT-in    WNT-in
4 1.477796e-02  1.1675655   1.0292063    1.322576     WNT-in    WNT-in
             id     cell_type   pvalue_adj label
1 NOTCH-in day3 proliferative 1.000000e+00      
2 NOTCH-in day7 proliferative 1.454251e-13   ***
3   WNT-in day3 proliferative 1.000000e+00      
4   WNT-in day7 proliferative 5.911185e-02      


Saving 6.67 x 6.67 in image

inter_PGRneg



        pvalue odds.ratio min_confint max_confint inhibitor2 inhibitor
1 1.910703e-08   1.274949    1.170179    1.389371   NOTCH-in  NOTCH-in
2 1.073089e-05   2.480641    1.632425    3.773022   NOTCH-in  NOTCH-in
3 2.951736e-04   1.173410    1.075509    1.280342     WNT-in    WNT-in
4 9.593339e-12   4.092886    2.703563    6.207404     WNT-in    WNT-in
             id    cell_type   pvalue_adj label
1 NOTCH-in day3 inter_PGRneg 7.642811e-08   ***
2 NOTCH-in day7 inter_PGRneg 4.292357e-05   ***
3   WNT-in day3 inter_PGRneg 1.180694e-03    **
4   WNT-in day7 inter_PGRneg 3.837336e-11   ***


Saving 6.67 x 6.67 in image

Warning message:
“Removed 4 rows containing missing values (geom_bar).”
KRT5_LRG6



         pvalue odds.ratio min_confint max_confint inhibitor2 inhibitor
1  2.569114e-01  0.9094673   0.7734760    1.069259   NOTCH-in  NOTCH-in
2 4.993927e-246  6.7639283   5.9875629    7.653755   NOTCH-in  NOTCH-in
3  9.049794e-01  0.9904692   0.8447003    1.161526     WNT-in    WNT-in
4  1.363901e-01  0.8322765   0.6491982    1.057164     WNT-in    WNT-in
             id cell_type    pvalue_adj label
1 NOTCH-in day3 KRT5_LRG6  1.000000e+00      
2 NOTCH-in day7 KRT5_LRG6 1.997571e-245   ***
3   WNT-in day3 KRT5_LRG6  1.000000e+00      
4   WNT-in day7 KRT5_LRG6  5.455605e-01      


Saving 6.67 x 6.67 in image

inter



        pvalue odds.ratio min_confint max_confint inhibitor2 inhibitor
1 2.743768e-02  1.4437525   1.0288005   2.0392091   NOTCH-in  NOTCH-in
2 7.027879e-02  1.0997190   0.9919397   1.2183814   NOTCH-in  NOTCH-in
3 1.771513e-17  3.2218440   2.3953850   4.3874077     WNT-in    WNT-in
4 3.442183e-55  0.2256796   0.1775486   0.2835591     WNT-in    WNT-in
             id cell_type   pvalue_adj label
1 NOTCH-in day3     inter 1.097507e-01      
2 NOTCH-in day7     inter 2.811151e-01      
3   WNT-in day3     inter 7.086054e-17   ***
4   WNT-in day7     inter 1.376873e-54   ***


Saving 6.67 x 6.67 in image

